# Crime stage


In [ ]:
import pandas as pd
import re
from sklearn.metrics import accuracy_score


sentences_df = pd.read_csv('/content/kr - Лист12.csv', header=None)
texts = sentences_df.iloc[:, 1]
labels_df = pd.read_csv('/content/Характеристики - общее_.csv')


def clean_text(text):
    if pd.isna(text):
        return ''
    text = str(text).lower()
    text = re.sub(r'[^\w\sстч\.]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


texts = texts.apply(clean_text)

def detect_crime_stage(text):
    text_normalized = re.sub(r'\s+', '', text)
    if re.search(r'ч\.?3ст\.?30', text_normalized):
        return 'покушение'
    else:
        return 'завершенное'


predicted_results = texts.apply(detect_crime_stage)


min_len = min(len(predicted_results), len(labels_df))
predicted_results = predicted_results.iloc[:min_len]
true_results = labels_df['Crime stage '].fillna('завершенное').str.lower().str.strip().iloc[:min_len]


accuracy = accuracy_score(true_results, predicted_results)
print(f'Accuracy: {accuracy:.4f}')


comparison_df = pd.DataFrame({
    'Текст': texts.iloc[:min_len],
    'Реальный результат': true_results,
    'Предсказанный результат': predicted_results
})

errors = comparison_df[comparison_df['Реальный результат'] != comparison_df['Предсказанный результат']]
print(f'Ошибок: {len(errors)} из {len(comparison_df)}')


print("\nТипичные ошибки:")
error_types = errors.groupby(['Реальный результат', 'Предсказанный результат']).size().sort_values(ascending=False)
print(error_types.head(10))


pd.set_option('display.max_colwidth', 200)
print("\nПримеры текстов с ошибками:")
print(errors[['Текст', 'Реальный результат', 'Предсказанный результат']].head(10))


Accuracy: 0.9408
Ошибок: 9 из 152

Типичные ошибки:
Реальный результат  Предсказанный результат
завершенное         покушение                  5
покушение           завершенное                4
dtype: int64

Примеры текстов с ошибками:
                                                                                                                                                                                                       Текст  \
52   судебный акт 1 приговор п р и г о в о р и м е н е м р о с с и й с к о й ф е д е р а ц и и г. астрахань 01 марта 2019 г. кировский районный суд г. астрахани в составе председательствующего судьи се...   
54   судебный акт 1 приговор п р и г о в о р и м е н е м р о с с и й с к о й ф е д е р а ц и и г. астрахань 13 февраля 2019 г. кировский районный суд г.астрахани в составе председательствующего судьи с...   
55   судебный акт 1 приговор п р и г о в о р и м е н е м р о с с и й с к о й ф е д е р а ц и и г. астрахань дд.мм.гггг г. кировский районный

# **drug_amount**

In [ ]:
import pandas as pd
import re
from sklearn.metrics import accuracy_score

# Загрузка данных
sentences_df = pd.read_csv('/content/kr - Лист12.csv', header=None)
texts = sentences_df.iloc[:, 1]
labels_df = pd.read_csv('/content/Характеристики - общее_.csv')

# Очистка текста
def clean_text(text):
    if pd.isna(text):
        return ''
    text = str(text).lower()
    text = re.sub(r'[^\w\sстч\.]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

texts = texts.apply(clean_text)

# Паттерны для статей УК
article_228_1_pattern = r'ч\.?\s*1\s*ст\.?\s*228\s*ук'
article_228_2_pattern = r'ч\.?\s*2\s*ст\.?\s*228\s*ук'

# Паттерны для ключевых слов
significant_pattern = r'значительн(ый|ом|ого|ые|ыми|ом)'
large_pattern = r'крупн(ый|ом|ого|ые|ыми|ом)'

# Функция для обработки текста
def detect_drug_amount(text):
    if pd.isna(text):
        return 'не определено'

    text = text.lower()
    text = re.sub(r'\s+', ' ', text)

    # Приоритет - статьи УК
    if re.search(article_228_1_pattern, text):
        return 'значительный'
    elif re.search(article_228_2_pattern, text):
        return 'крупный'

    # Смотрим на ключевые слова
    if re.search(large_pattern, text):
        return 'крупный'
    elif re.search(significant_pattern, text):
        return 'значительный'

    # Если ничего не нашли
    return 'не определено'

# Применяем улучшенную функцию
predicted_drug_amount = texts.apply(detect_drug_amount)

# Выравниваем длины
min_len = min(len(predicted_drug_amount), len(labels_df))
predicted_drug_amount = predicted_drug_amount.iloc[:min_len]
true_drug_amount = labels_df['drug_amount'].fillna('не определено').str.lower().str.strip().iloc[:min_len]

# Accuracy
accuracy = accuracy_score(true_drug_amount, predicted_drug_amount)
print(f'Accuracy: {accuracy:.4f}')

# Анализ ошибок
comparison_df = pd.DataFrame({
    'Текст': texts.iloc[:min_len],
    'Реальный результат': true_drug_amount,
    'Предсказанный результат': predicted_drug_amount
})

errors = comparison_df[comparison_df['Реальный результат'] != comparison_df['Предсказанный результат']]
print(f'Ошибок: {len(errors)} из {len(comparison_df)}')

# Ошибки
print("\nТипичные ошибки:")
error_types = errors.groupby(['Реальный результат', 'Предсказанный результат']).size().sort_values(ascending=False)
print(error_types.head(10))

# Примеры
pd.set_option('display.max_colwidth', 200)
print("\nПримеры текстов с ошибками:")
print(errors[['Текст', 'Реальный результат', 'Предсказанный результат']].head(10))


Accuracy: 0.5987
Ошибок: 61 из 152

Типичные ошибки:
Реальный результат     Предсказанный результат
значительный           крупный                    35
крупный                значительный               19
значительный           не определено               2
не определено          значительный                2
значительный, крупный  значительный                1
крупный                не определено               1
не определено          крупный                     1
dtype: int64

Примеры текстов с ошибками:
                                                                                                                                                                                                      Текст  \
0                                                                                                                                                                                                             
2   судебный акт 1 приговор приговор именем российской федерации 10 сентя

# **Drug_type**

In [ ]:
import re
import pandas as pd
from sklearn.metrics import accuracy_score

kr = pd.read_csv('kr - Лист12-2.csv')
chars = pd.read_csv('Характеристики - общее_-2.csv')


kr = kr.rename(columns={kr.columns[0]: 'ID'})
chars = chars.rename(columns={chars.columns[0]: 'ID'})

kr['Clean_Text'] = kr['Text'].apply(lambda x: re.sub(r'\s+', ' ', str(x)).strip())

drug_synonyms = {
    "марихуана": [
        "марихуана", "конопля", "гашиш", "гашиш ", "каннабис", "каннабис; \nконопля",
        "масло каннабиса", "масло каннабиса ", "масло каннабиса;\n гашиш ",
        "гашишное масло"
    ],
    "pvp": ["PVP", "\nPVP"],
    "мефедрон": ["мефедрон", "Мефедрон"],
    "мдма": ["МДМА"],
    "спайс": ["спайс", "AB-PINACA-CHM", "АВ-PINACA-CHM"],
    "героин": ["героин"],
    "метадон": ["метадон"],
    "маковая солома": ["экстракт\nмаковой соломы "],
    "прочее": [
        "порошкообразное вещество", "порошкообразное вещество белого цвета",
        "тетраметилциклопропанкарбонил", "MDMB(N)-2201", "MDMB (N)-2201",
        "ТМСР-2201", "ММВА(N)-СНМ", "скорость"
    ],
    "не указано": ["nan"]
}


reverse_drug_synonyms = {v.strip().lower(): k for k, values in drug_synonyms.items() for v in values}


drug_pattern = r"(героин|марихуана|конопля|каннабис|гашиш(?:ное масло)?|масло каннабиса|pvp|мефедрон|мдма|спайс|AB-PINACA-CHM|АВ-PINACA-CHM|метадон|экстракт\s*маковой\s*соломы|MDMB\(N\)-2201|MDMB\s*\(N\)-2201|ТМСР-2201|ММВА\(N\)-СНМ|скорость|тетраметилциклопропанкарбонил|порошкообразное вещество(?: белого цвета)?)"

def extract_and_normalize_drug(text):
    if not isinstance(text, str):
        return 'не указано'

    match = re.search(drug_pattern, text, flags=re.IGNORECASE)
    if match:
        found_drug = match.group(1).strip().lower()
        normalized = reverse_drug_synonyms.get(found_drug, 'прочее')
        return normalized
    else:
        return 'не указано'

kr['Predicted_Drug'] = kr['Clean_Text'].apply(extract_and_normalize_drug)

merged = pd.merge(kr[['ID', 'Predicted_Drug']], chars[['ID', 'Drug_type']], on='ID', how='left')
merged = merged.dropna(subset=['Drug_type', 'Predicted_Drug'])


merged['Drug_type'] = merged['Drug_type'].astype(str).str.strip().str.lower()
merged['Predicted_Drug'] = merged['Predicted_Drug'].astype(str).str.strip().str.lower()



accuracy = accuracy_score(merged['Drug_type'], merged['Predicted_Drug'])

print(f'Accuracy: {accuracy:.4f}')



try:
    comparison_df = pd.DataFrame({
        'ID': merged['ID'],
        'Истинный наркотик': merged['Drug type'],
        'Предсказанный наркотик': merged['Predicted_Drug']
    })

    errors = comparison_df[comparison_df['Истинный наркотик'] != comparison_df['Предсказанный наркотик']]
    errors.to_csv('drug_type_errors.csv', index=False, encoding='utf-8-sig')
    print("\n✅ Ошибки сохранены в файл 'drug_type_errors.csv'")

except Exception as e:
    print(f"")


Accuracy: 0.7800



# sentence_years, sentence_months, sentence_days, sentence_hours, is_suspended, suspention_years, suspention_months, fine_is_fixed, fine_amount


In [ ]:
import pandas as pd
import re
from sklearn.metrics import accuracy_score

def parse_fine(text):
    result = {
        "Is_fixed_fine": 1,
        "Fine amount": None
    }

    fixed_patterns = [
        r"штраф[а-яё]*\s*(?:в\s*размере|)\s*(\d[\d\s]*(?=\s*\(|[\sруб]))",
        r"штраф[а-яё]*\s*(?:в\s*размере|)\s*(\d+)\s*(?:тыс|тысяч)",
        r"штраф[а-яё]*\s*в\s*размере\s*(\d[\d\s]*)\s*рубл",
        r"взыска(?:ть|ни[ея])\s*штраф\s*в?\s*размере\s*(\d[\d\s]*)",
    ]

    percent_patterns = [
        r"удержани(?:ем|ю)\s*(?:из|с)\s*(?:зарплаты|дохода)\s*(\d+)%?",
        r"(\d+)%\s*(?:от|из)\s*(?:заработка|дохода)",
        r"взыскани[ея]\s*(\d+)%?\s*(?:из|с)\s*заработка",
    ]

    if "штраф" not in text.lower() and "взыска" not in text.lower():
        return result

    for pattern in fixed_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            amount = match.group(1).replace(" ", "")
            if amount.isdigit():
                result["Is_fixed_fine"] = 1
                result["Fine amount"] = int(amount)
                return result

    for pattern in percent_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            percent = match.group(1)
            if percent.isdigit():
                result["Is_fixed_fine"] = 0
                result["Fine amount"] = f"{percent}%"
                return result

    return result

def parse_term(text):
    result = {
        "y": 0,
        "m": 0,
        "d": 0,
        "h": 0,
    }
    # Лет / годов / года
    year_match = re.search(r"\s(\d{1,2})(\s*\(.*\))?\s*(год|года|годов|лет)", text, re.DOTALL)
    if year_match:
        result["y"] = int(year_match.group(1))

    # Месяц / месяцев
    month_match = re.search(r"(\d+)(\s*\(.*\))?\s*(месяц(а|ев)?)", text)
    if month_match:
        result["m"] = int(month_match.group(1))

    # День / дней
    day_match = re.search(r"(\d+)(\s*\(.*\))?\s*(день|дня|дней)", text)
    if day_match:
        result["d"] = int(day_match.group(1))

    # Час / часов
    hour_match = re.search(r"(\d+)(\s*\(.*\))?\s*(час(ов)?)", text)
    if hour_match:
        result["h"] = int(hour_match.group(1))

    return result

def parse_sentence(text):
    result = {
        "Sentence year": 0,
        "Sentence month": 0,
        "Sentence day": 0,
        "Sentence hour": 0,
        "Is suspended": 0,
        "Suspention years": 0,
        "Suspention months": 0,
        "Is_fixed_fine": 0,
        "Fine amount": 0,
    }

    pattern_prigovoril = r"(п\s*р\s*и\s*г\s*о\s*в\s*о\s*р\s*и\s*л)[\s:,]+(.+)"
    match = re.search(pattern_prigovoril, text.lower(), re.DOTALL)
    if not match:
        return result

    sentence_text = match.group(2)

    suspended_patterns = [
        "наказание считать условным",
        "условным осуждением",
        "с испытательным сроком",
        "в соответствии со ст\\. 73 ук рф",
        "статья 73 ук рф"
    ]

    suspention_index = None
    for pattern in suspended_patterns:
        match = re.search(pattern, sentence_text, re.IGNORECASE)
        if match:
            result["Is suspended"] = 1
            suspention_index = match.start()
            term = parse_term(sentence_text[suspention_index:])
            result["Suspention years"] = term['y']
            result["Suspention months"] = term['m']
            break

    if suspention_index is None:
        term_text = sentence_text
    else:
        term_text = sentence_text[:suspention_index]

    sentence_term = parse_term(term_text)
    result["Sentence year"]  = sentence_term["y"]
    result["Sentence month"] = sentence_term["m"]
    result["Sentence day"]   = sentence_term["d"]
    result["Sentence hour"]  = sentence_term["h"]


    fine_data = parse_fine(sentence_text)
    result["Is_fixed_fine"] = fine_data["Is_fixed_fine"]
    result["Fine amount"] = fine_data["Fine amount"]

    return result


file_path = 'kr - Лист12-4.csv'
df = pd.read_csv(file_path)
if 'Text' not in df.columns:
    raise ValueError("В файле нет колонки 'Text'!")

parsed_results = df['Text'].apply(parse_sentence)
parsed_df = pd.DataFrame(parsed_results.tolist())
final_df = pd.concat([df, parsed_df], axis=1)
final_df.to_csv('parsed_sentences.csv', index=False, encoding='utf-8-sig')
print("Готово! Файл сохранен как 'parsed_sentences.csv'.")

parsed_df = pd.read_csv('parsed_sentences.csv')
true_labels_df = pd.read_csv('Характеристики - общее_-4.csv')

required_columns = [
    "Sentence year", "Sentence month", "Sentence day", "Sentence hour",
    "Is suspended", "Suspention years", "Suspention months",
    "Is_fixed_fine", "Fine amount"
]

for col in required_columns:
    if col not in parsed_df.columns:
        raise ValueError(f"В parsed_sentences.csv нет колонки {col}")
    if col not in true_labels_df.columns:
        raise ValueError(f"В Характеристики - общее_-4.csv нет колонки {col}")

accuracies = {}

for col in required_columns:
    y_pred = parsed_df[col].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
    y_true = true_labels_df[col].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

    acc = accuracy_score(y_true, y_pred)
    accuracies[col] = acc

print("Accuracy по каждому столбцу:")
for col, acc in accuracies.items():
    print(f"{col}: {acc:.4f}")


Готово! Файл сохранен как 'parsed_sentences.csv'.
Accuracy по каждому столбцу:
Sentence year: 0.8800
Sentence month: 0.9267
Sentence day: 0.9133
Sentence hour: 0.9467
Is suspended: 0.9600
Suspention years: 0.9067
Suspention months: 0.9800
Is_fixed_fine: 0.9600
Fine amount: 0.9333
